In [ ]:
from rdflib import Graph, Namespace, Literal
from rdflib.namespace import RDF, RDFS, NamespaceManager
from urllib.parse import urlparse


g = Graph()
n = Graph()
g.parse("20140114.rdf")
SCH = Namespace("http://xmlns.com/foaf/0.1/")
DGRAPH = Namespace("dgraph.")
NEW = Namespace("dgraphnew.")
BLANK = Namespace("_:")

print(len(g))


# prints: 2
def parse_ref(uri):
    parsed_url = urlparse(uri)
    if parsed_url.fragment == "":
        namespace = f"{parsed_url.scheme}://{parsed_url.netloc}{parsed_url.path.rsplit('/', 1)[0]}/"
        term = parsed_url.path.split("/")[-1]
    else:
        namespace = f"{parsed_url.scheme}://{parsed_url.netloc}{parsed_url.path}"
        term = parsed_url.fragment
    return (namespace, term)


# find all subjects of any type
n.add((BLANK.test, RDFS.label, Literal(SCH)))
n.add((BLANK.test, DGRAPH["type"], Literal("dgraph.ontology")))
for subject, p, o in g.triples((None, RDF.type, RDFS.Class)):
    (namespace, term) = parse_ref(subject)

    if namespace == SCH:
        # search the label
        label = g.value(subject, RDFS.label)

        n.add((BLANK.test, NEW.Class, BLANK[subject]))
        n.add((BLANK[subject], RDFS.label, Literal(str(label))))
        for s1, p1, o1 in g.triples((subject, RDFS.subClassOf, None)):
            n.add((BLANK[subject], p1, BLANK[o1]))
for subject, p, o in g.triples((None, RDF.type, RDF.Property)):
    (namespace, term) = parse_ref(subject)

    if namespace == SCH:
        # search the label
        label = g.value(subject, RDFS.label)

        n.add((BLANK.test, NEW.Property, BLANK[subject]))
        n.add((BLANK[subject], RDFS.label, Literal(str(label))))
        for s1, p1, o1 in g.triples((subject, RDFS.subClassOf, None)):
            n.add((BLANK[subject], p1, BLANK[o1]))
nquads = n.serialize(format="nt")

631


In [ ]:
# Approach 2 : using isDefinedBy
from rdflib import Graph, Namespace, Literal
from rdflib.namespace import RDF, RDFS, NamespaceManager
from urllib.parse import urlparse


g = Graph()
n = Graph()
g.parse("20140114.rdf")
SCH = Namespace("http://xmlns.com/foaf/0.1/")
DGRAPH = Namespace("dgraph.")
NEW = Namespace("dgraphnew.")
BLANK = Namespace("_:")

print(len(g))


# prints: 2
def parse_ref(uri):
    parsed_url = urlparse(uri)
    if parsed_url.fragment == "":
        namespace = f"{parsed_url.scheme}://{parsed_url.netloc}{parsed_url.path.rsplit('/', 1)[0]}/"
        term = parsed_url.path.split("/")[-1]
    else:
        namespace = f"{parsed_url.scheme}://{parsed_url.netloc}{parsed_url.path}"
        term = parsed_url.fragment
    return (namespace, term)


# find all subjects of any type
n.add((BLANK.test, RDFS.label, Literal(SCH)))
n.add((BLANK.test, DGRAPH["type"], Literal("dgraph.ontology")))
for subject, p, o in g.triples((None, RDFS.isDefinedBy, None)):
    (namespace, term) = parse_ref(subject)
    print(f"namespace: {namespace} term: {term} - defined by: {str(o)}")
    for s1, p1, o1 in g.triples((subject, None, None)):
        print(f": {p1}  {o1}")
        # n.add((BLANK[subject], p1,BLANK[o1]))

nquads = n.serialize(format="nt")

631
namespace: http://xmlns.com/foaf/0.1/ term: LabelProperty - defined by: http://xmlns.com/foaf/0.1/
: http://www.w3.org/1999/02/22-rdf-syntax-ns#type  http://www.w3.org/2000/01/rdf-schema#Class
: http://www.w3.org/1999/02/22-rdf-syntax-ns#type  http://www.w3.org/2002/07/owl#Class
: http://www.w3.org/2003/06/sw-vocab-status/ns#term_status  unstable
: http://www.w3.org/2000/01/rdf-schema#label  Label Property
: http://www.w3.org/2000/01/rdf-schema#comment  A foaf:LabelProperty is any RDF property with texual values that serve as labels.
: http://www.w3.org/2000/01/rdf-schema#isDefinedBy  http://xmlns.com/foaf/0.1/
namespace: http://xmlns.com/foaf/0.1/ term: Person - defined by: http://xmlns.com/foaf/0.1/
: http://www.w3.org/1999/02/22-rdf-syntax-ns#type  http://www.w3.org/2000/01/rdf-schema#Class
: http://www.w3.org/1999/02/22-rdf-syntax-ns#type  http://www.w3.org/2002/07/owl#Class
: http://www.w3.org/2000/01/rdf-schema#label  Person
: http://www.w3.org/2000/01/rdf-schema#comment  A p

In [136]:
import pydgraph

dgraph_grpc = "localhost:9080"
api_key = ""
# dgraph_grpc = "blue-surf-1000009.grpc.us-east-1.aws.cloud.dgraph.io:443"
if "cloud.dgraph.io" in dgraph_grpc:
    client_stub = pydgraph.DgraphClientStub.from_cloud(dgraph_grpc, api_key)
else:
    client_stub = pydgraph.DgraphClientStub(dgraph_grpc)
client = pydgraph.DgraphClient(client_stub)

# Drop all data including schema from the Dgraph instance. This is a useful
# for small examples such as this since it puts Dgraph into a clean state.

op = pydgraph.Operation(drop_op=pydgraph.Operation.DropOp.DATA)
client.alter(op)
print("data deleted")
txn = client.txn()
xidmap = dict({})
try:
    # nRun query.
    res = txn.mutate(set_nquads=nquads)
    txn.commit()
    for n in res.uids:
        xidmap["<_:" + n + ">"] = "<" + res.uids[n] + ">"
finally:
    txn.discard()
print(xidmap)

data deleted
{'<_:http://xmlns.com/foaf/0.1/OnlineEcommerceAccount>': '<0x1b81053>', '<_:http://xmlns.com/foaf/0.1/homepage>': '<0x1b8106d>', '<_:http://www.w3.org/2002/07/owl#Thing>': '<0x1b8105e>', '<_:http://xmlns.com/foaf/0.1/topic>': '<0x1b81084>', '<_:http://xmlns.com/foaf/0.1/familyName>': '<0x1b81067>', '<_:http://xmlns.com/foaf/0.1/sha1>': '<0x1b8108a>', '<_:http://xmlns.com/foaf/0.1/gender>': '<0x1b8108b>', '<_:http://xmlns.com/foaf/0.1/lastName>': '<0x1b81072>', '<_:http://xmlns.com/foaf/0.1/nick>': '<0x1b81047>', '<_:http://xmlns.com/foaf/0.1/Group>': '<0x1b81044>', '<_:http://xmlns.com/foaf/0.1/tipjar>': '<0x1b8106c>', '<_:http://xmlns.com/foaf/0.1/publications>': '<0x1b8105b>', '<_:http://xmlns.com/foaf/0.1/givenname>': '<0x1b81045>', '<_:http://xmlns.com/foaf/0.1/jabberID>': '<0x1b81083>', '<_:http://xmlns.com/foaf/0.1/surname>': '<0x1b81061>', '<_:http://xmlns.com/foaf/0.1/knows>': '<0x1b8107f>', '<_:http://xmlns.com/foaf/0.1/workplaceHomepage>': '<0x1b8107e>', '<_:http

In [86]:
from rdflib import Graph, Namespace
from rdflib.namespace import RDF, RDFS
from urllib.parse import urlparse

g = Graph()
n = Graph()
g.parse("schemaorg-current-https.nt")
SCH = Namespace("https://schema.org/")
BLANK = Namespace("_:")
print(len(g))
# prints: 2

# find all subjects of any type
for subject, p, o in g.triples((None, RDFS.label, None)):
    (namespace, term) = parse_ref(subject)
    if namespace == SCH:
        print(subject, p, o)

16776
https://schema.org/Discontinued http://www.w3.org/2000/01/rdf-schema#label Discontinued
https://schema.org/DigitalArtDigitalSource http://www.w3.org/2000/01/rdf-schema#label DigitalArtDigitalSource
https://schema.org/VitalSign http://www.w3.org/2000/01/rdf-schema#label VitalSign
https://schema.org/usageInfo http://www.w3.org/2000/01/rdf-schema#label usageInfo
https://schema.org/featureList http://www.w3.org/2000/01/rdf-schema#label featureList
https://schema.org/model http://www.w3.org/2000/01/rdf-schema#label model
https://schema.org/resultComment http://www.w3.org/2000/01/rdf-schema#label resultComment
https://schema.org/Beach http://www.w3.org/2000/01/rdf-schema#label Beach
https://schema.org/NightClub http://www.w3.org/2000/01/rdf-schema#label NightClub
https://schema.org/RentAction http://www.w3.org/2000/01/rdf-schema#label RentAction
https://schema.org/Movie http://www.w3.org/2000/01/rdf-schema#label Movie
https://schema.org/category http://www.w3.org/2000/01/rdf-schema#lab

In [129]:
f = Graph()
f.parse("20140114.rdf")


print(len(g))


# prints: 2
def parse_ref(uri):
    parsed_url = urlparse(uri)
    if parsed_url.fragment == "":
        namespace = f"{parsed_url.scheme}://{parsed_url.netloc}{parsed_url.path.rsplit('/', 1)[0]}/"
        term = parsed_url.path.split("/")[-1]
    else:
        namespace = f"{parsed_url.scheme}://{parsed_url.netloc}{parsed_url.path}"
        term = parsed_url.fragment
    return (namespace, term)


# find all subjects of any type

for subject, p, o in f.triples((None, None, RDF.Property)):
    print(subject, p, o)

631
http://xmlns.com/foaf/0.1/mbox http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://www.w3.org/1999/02/22-rdf-syntax-ns#Property
http://xmlns.com/foaf/0.1/mbox_sha1sum http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://www.w3.org/1999/02/22-rdf-syntax-ns#Property
http://xmlns.com/foaf/0.1/gender http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://www.w3.org/1999/02/22-rdf-syntax-ns#Property
http://xmlns.com/foaf/0.1/geekcode http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://www.w3.org/1999/02/22-rdf-syntax-ns#Property
http://xmlns.com/foaf/0.1/dnaChecksum http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://www.w3.org/1999/02/22-rdf-syntax-ns#Property
http://xmlns.com/foaf/0.1/sha1 http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://www.w3.org/1999/02/22-rdf-syntax-ns#Property
http://xmlns.com/foaf/0.1/based_near http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://www.w3.org/1999/02/22-rdf-syntax-ns#Property
http://xmlns.com/foaf/0.1/title http://www.w3.org/19

In [134]:
g.namespace_manager.namespaces

<bound method NamespaceManager.namespaces of <rdflib.namespace.NamespaceManager object at 0x10c6d5290>>